<a href="https://colab.research.google.com/github/captainArrcus/Audifine/blob/master/Data_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
  This Notebook exists to generate some own datapoints. 
  In the label_list I can put all the words, I want spoken datapoints to.

  To get some more points, I mixxed the examples with different backgroundnoises. 
'''

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pydub

In [ ]:
!apt remove libav-tools
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio

In [ ]:
#Function to get soundexamples


import pyaudio
import wave
def soundexample(time, filename):
    chunk = 1024  # Record in chunks of 1024 samples
    sample_format = pyaudio.paInt16  # 16 bits per sample
    channels = 1
    fs = 16000  # Record at 16000 samples per second because other soundexamples have the same
    seconds = time
   # filename = filename

    p = pyaudio.PyAudio()  # Create an interface to PortAudio

    print('Recording')

    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)

    frames = []  # Initialize array to store frames

    # Store data in chunks for 3 seconds
    for i in range(0, int(fs / chunk * seconds)):
        data = stream.read(chunk)
        frames.append(data)

    # Stop and close the stream 
    stream.stop_stream()
    stream.close()
    # Terminate the PortAudio interface
    p.terminate()

    print('Finished recording')

    # Save the recorded data as a WAV file
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()


In [ ]:
#Function to split words

# Import the AudioSegment class for processing audio and the 
# split_on_silence function for separating out silent chunks.
from pydub import AudioSegment
from pydub.silence import split_on_silence


def split_in_single_words(input_file, input_path, output_path):
    input_file_wav = input_path + input_file + ".wav"
    
    # Load your audio.
    song = AudioSegment.from_wav(input_file_wav)

    # Split track where the silence is 2 seconds or more and get chunks using 
    # the imported function.
    chunks = split_on_silence (
        # Use the loaded audio.
        song, 
        # Specify that a silent chunk must be at least 2 seconds or 2000 ms long.
        min_silence_len = 200,
        # Consider a chunk silent if it's quieter than -16 dBFS.
        # (You may want to adjust this parameter.)
        silence_thresh = -16
    )
    
    

    # Process each chunk with your parameters
    for i, chunk in enumerate(chunks):
        
        # Create output path
        output = output_path + "/"+input_file+"{0}.wav"
        
        # Create a silence chunk that's 0.5 seconds (or 200 ms) long for padding.
        silence_chunk = AudioSegment.silent(duration=200)

        # Add the padding chunk to beginning and end of the entire chunk.
        audio_chunk = silence_chunk + chunk + silence_chunk

        # Normalize the entire chunk.
        normalized_chunk = match_target_amplitude(audio_chunk, -20.0)

        # Export the audio chunk with new bitrate.
        
        print("Exporting "+output+"{0}.wav.".format(i))
        normalized_chunk.export(
           output.format(i),
            bitrate = "192k",
            format = "wav"
        )

In [ ]:
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

In [ ]:
#Mix my trainingsset up with some backgroundnoise

#!!! Outputpath should NOT be file_directory or augmentation-directory, else INFINITILOOP
from pydub import AudioSegment
import os
import random

def data_augmentation(augmentation_directory, file_directory, outputpath):
    #Iterate over all augmentation files
    for aug_file in os.listdir(augmentation_directory):
        if aug_file.endswith(".wav"): 
            
            # Iterate over all Trainingexamples
            for filename in os.listdir(file_directory):
                if filename.endswith(".wav"): 
                    #mix the two wav-files
                    trainings_example = AudioSegment.from_file(file_directory + filename)
                    augmentation = AudioSegment.from_file(augmentation_directory + aug_file)

                    #randomize the backgroundnoise a little
                    augmentation = augmentation[(random.randint(1,len(augmentation))):]
                    augmentation = match_target_amplitude(augmentation,(random.randint(-60,-1)))
                    #puts a random sequence from the augmented file on the sample
                    combined = trainings_example.overlay(augmentation,gain_during_overlay = 2)
                    #AudioSegment.overlay()
                    #Get the of the sample right
                    base = os.path.basename(filename)
                    namebase = os.path.splitext(base)[0]
                    
                    base_aug = os.path.basename(aug_file)
                    namebase_aug = os.path.splitext(base_aug)[0]
                    
                    outputfile = outputpath + namebase +'__'+ namebase_aug + '.wav'
                    print(outputfile)
                    combined.export(outputfile, format='wav')
                    continue
    #Now bring all the data together
   # for file_name in file_directory:        
    #    shutil.move(os.path.join(file_directory, file_name), outputpath)

In [ ]:
trainings_example = AudioSegment.from_file('/content/drive/MyDrive/Audifine/Debug/Trainingsdata/Kopie von Audifine1.wav')
augmentation = AudioSegment.from_file('/content/drive/MyDrive/Audifine/Debug/BackgroundNoise/Kopie von Home-Phone-Ringing.wav')
#puts a random sequence from the augmented file on the sample
augmentation =augmentation[(random.randint(1,len(augmentation))):]
combined = trainings_example.overlay(augmentation, gain_during_overlay =2)
print(random.randint(-60,-1))
normalized_sound = match_target_amplitude(combined, random.randint(1,60))
normalized_sound

In [ ]:
data_augmentation('/content/drive/MyDrive/Audifine/Debug/BackgroundNoise/','/content/drive/MyDrive/Audifine/Debug/Trainingsdata/','/content/drive/MyDrive/Audifine/Debug/')

In [ ]:
#Correct Data Error
import librosa
import soundfile as sf
import numpy as np
def correct_wav_files(file_directory):
  for filename in os.listdir(file_directory):
    if filename.endswith(".wav"): 
      file_path = file_directory + filename
      bug_data,_ = librosa.load(file_path,mono=True, sr=16000, dtype = np.float32)

      new_file_path = file_path + 'small'
      sf.write(file_path,bug_data, 16000)

In [ ]:
bug_data= librosa.load('/content/drive/MyDrive/Audifine/_background_noise_/Home-Phone-Ringing.wav')
print(bug_data)

In [ ]:
# Sort data, so that you can work with the tensorflow tutorial
import shutil
import os


def data_split_by_name(training_path,label_list):
    file_names = os.listdir(training_path)
    for word in label_list:
        path = training_path + word
        #os.makedirs(path, exist_ok=True)  // exits_ok doesn't work for me
        if not os.path.isdir(path):
            os.makedirs(path)
        for file_name in file_names:
            if file_name.startswith(word):
                shutil.move(os.path.join(training_path, file_name), path)

In [ ]:
import time
#soundexample(9, "output.wav")



#Get a List of all Words, you want to get examples from
label_list = ['Transformer','Neural Network','CNN','Audifine','Spracherkennung','Python','Layers','Preprocessing','Tensorflow','Programmieren']

In [ ]:
path = 'train_data'
time_to_say_an_example = 50
for example in label_list:
    filename = example +'.wav'
    print('Jetzt ' + example + ' sagen')
    time.sleep(5)
    soundexample(time_to_say_an_example, filename)

In [ ]:
correct_wav_files('/content/drive/MyDrive/Audifine/')
correct_wav_files('/content/drive/MyDrive/Audifine/_background_noise_/')

In [ ]:
#Extract single Words in my recorded Examples
for example in label_list:
    
    split_in_single_words(example,'/content/drive/MyDrive/Audifine/', '/content/drive/MyDrive/Audifine/Trainingsdaten/')
    


In [ ]:
data_augmentation('/content/drive/MyDrive/Audifine/_background_noise_/','/content/drive/MyDrive/Audifine/Trainingsdaten/','/content/drive/MyDrive/Audifine/Train/')

In [ ]:
data_split_by_name('/content/drive/MyDrive/Audifine/Train/',label_list)


In [ ]:
#Data exploration because, theres something wrong
import wave
import contextlib
from glob import glob

maxframes = 0
minframes = 10000000000 
for i in glob("/content/drive/MyDrive/Audifine/Train/*"):
  
  if i.endswith(".wav"):

    fname = i 
    with contextlib.closing(wave.open(fname,'r')) as f:
      frames = f.getnframes()
      rate = f.getframerate()
      duration = frames / float(rate)
      #print(str(duration) + i)
      if maxframes<frames:
        maxframes=frames
      if minframes>frames:
        minframes=frames
print('Die meisten Frames %s' % maxframes)
print('Die wenigsten Frames %s' % minframes)